# YouTube Music

# Contenidos <a id='back'></a>

* [Introducción](#intro)

* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>

En este proyecto compararemos las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiaremos datos reales de YouTube.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo: 
Probar tres **hipótesis**: 

- **1.- La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.**

- **2.- Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche.**

- **3.- Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.**

### Etapas 
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`.

Primero, evaluaremos la calidad de los datos y veremos si los problemas son significativos. 
Entonces, durante el preprocesamiento de datos tendremos en cuenta los problemas más críticos.
 
El proyecto consistirá en tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis
 
[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>


In [25]:
import pandas as pd

In [26]:
# leyendo el archivo y almacenándolo en la variable df
df = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/music_project_en.csv")
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


Imprimimos las 10 primeras filas de la tabla:

In [27]:
# obteniendo las 10 primeras filas de la tabla df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Obtenemos la información general sobre la tabla con un comando:

In [28]:
# obteniendo información general sobre los datos en df
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. 
Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — la hora exacta en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. La columna 'userID' deberia estar escrita en snake_case así: user_id(cada palabra separada con un guion bajo)


El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.Porque hay 65079 filas.


### Conclusiones (Etapa 1) <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista. 

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>


Vamos a corregir el formato en los encabezados de las columnas y nos ocuparemos de los valores ausentes, al final revisaremos si hay duplicados en los datos.

### Estilo del encabezado <a id='header_style'></a>
Imprimiremos el encabezado de las columnas:

In [29]:
# la lista de los nombres de las columnas en la tabla df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Cambiaremos los nombres de las columnas de acuerdo con las reglas del buen estilo:
* Si el nombre tiene varias palabras, utilizaremos snake_case
* Todos los caracteres deben ser minúsculas
* Eliminaremos los espacios

In [30]:
df.columns = df.columns.str.lower()

In [31]:
df.columns = df.columns.str.strip()


Comprobaremos  el resultado. Imprimiremos los nombres de las columnas una vez más:

In [32]:
# comprobando el resultado: la lista de los nombres de las columnas
df.columns

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Valores ausentes <a id='missing_values'></a>
Utilizaremos dos métodos de pandas para encontrar valores ausentes

In [33]:
# calculando valores ausentes
df.isna().sum()

userid       0
track     1343
artist    7567
genre     1198
city         0
time         0
day          0
dtype: int64

No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en 'track' y 'artist' no son cruciales.Podemos reemplazarlos por marcadores claros.

Pero los valores ausentes en `'genre'` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendremos que:
* Rellenar esos valores ausentes con marcadores
* Evaluar cuánto podrían afectar los valores ausentes 

Reemplazaremos los valores ausentes en `'track'`, `'artist'`, y `'genre'` con la string `'unknown'`. Para ello, crearemos la lista `columns_to_replace`, la recorreremos con un bucle `for` y reemplazaremos los valores ausentes en cada una de las columnas:

In [34]:
# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column]= df[column].fillna('unknow')
        

In [40]:
# contando valores ausentes
df.isna().sum()

userid    0
track     0
artist    0
genre     0
city      0
time      0
day       0
dtype: int64

### Duplicados <a id='duplicates'></a>


In [41]:
# contando duplicados obvios
df.duplicated().sum()

0

Llama al método `pandas` para deshacerte de los duplicados obvios:

In [42]:
# eliminando duplicados obvios y resetearemos el indice para llevar un orden en las filas
df = df.drop_duplicates().reset_index(drop=True)

Contando los duplicados obvios una vez más para asegurarnos de que todos han sido eliminados:

In [43]:
# comprobando duplicados
df.duplicated().sum()

0

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userid  61253 non-null  object
 1   track   61253 non-null  object
 2   artist  61253 non-null  object
 3   genre   61253 non-null  object
 4   city    61253 non-null  object
 5   time    61253 non-null  object
 6   day     61253 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


Ahora nos desharemos de los duplicados implícitos en la columna `'genre'`.
Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar al resultado.

Imprimiremos una lista de nombres únicos de géneros, ordenados en orden alfabético.


In [39]:
# inspeccionando los nombres de géneros únicos
df_unique = sorted(df['genre'].unique())
for genre in df_unique:
    print(genre)


acid
acoustic
action
adult
africa
afrikaans
alternative
ambient
americana
animated
anime
arabesk
arabic
arena
argentinetango
art
audiobook
avantgarde
axé
baile
balkan
beats
bigroom
black
bluegrass
blues
bollywood
bossa
brazilian
breakbeat
breaks
broadway
cantautori
cantopop
canzone
caribbean
caucasian
celtic
chamber
children
chill
chinese
choral
christian
christmas
classical
classicmetal
club
colombian
comedy
conjazz
contemporary
country
cuban
dance
dancehall
dancepop
dark
death
deep
deutschrock
deutschspr
dirty
disco
dnb
documentary
downbeat
downtempo
drum
dub
dubstep
eastern
easy
electronic
electropop
emo
entehno
epicmetal
estrada
ethnic
eurofolk
european
experimental
extrememetal
fado
film
fitness
flamenco
folk
folklore
folkmetal
folkrock
folktronica
forró
frankreich
französisch
french
funk
future
gangsta
garage
german
ghazal
gitarre
glitch
gospel
gothic
grime
grunge
gypsy
handsup
hard'n'heavy
hardcore
hardstyle
hardtechno
hip
hip-hop
hiphop
historisch
holiday
hop
horror
house
idm
i

En la lista hay duplicados implícitos del género `hiphop`. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

* `hip`
* `hop`
* `hip-hop`

Para deshacernos de ellos, declaramos la función `replace_wrong_genres()` con dos parámetros: 
* `wrong_genres=` — la lista de duplicados
* `correct_genre=` — la string con el valor correcto

La función debería corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, remplazaremos cada valor de la lista `wrong_genres` con el valor en `correct_genre`.

In [46]:
# función para reemplazar duplicados implícitos
def replace_wrong_genres(wrong_genres, correct_genre):
    df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

Llamaremos a nuestra función `replace_wrong_genres()` y pasaremos los argumentos para que retire los duplicados implícitos (`hip`, `hop` y `hip-hop`) 

In [47]:
# eliminando duplicados implícitos
replace_wrong_genres(['hip', 'hop', 'hip-hop'], "hiphop")


Nos aseguramos que los nombres duplicados han sido eliminados. 

In [48]:
# revisando en busca de duplicados implícitos
df_unique =  sorted(df['genre'].unique())
for genre in df_unique:
    print(genre)

acid
acoustic
action
adult
africa
afrikaans
alternative
ambient
americana
animated
anime
arabesk
arabic
arena
argentinetango
art
audiobook
avantgarde
axé
baile
balkan
beats
bigroom
black
bluegrass
blues
bollywood
bossa
brazilian
breakbeat
breaks
broadway
cantautori
cantopop
canzone
caribbean
caucasian
celtic
chamber
children
chill
chinese
choral
christian
christmas
classical
classicmetal
club
colombian
comedy
conjazz
contemporary
country
cuban
dance
dancehall
dancepop
dark
death
deep
deutschrock
deutschspr
dirty
disco
dnb
documentary
downbeat
downtempo
drum
dub
dubstep
eastern
easy
electronic
electropop
emo
entehno
epicmetal
estrada
ethnic
eurofolk
european
experimental
extrememetal
fado
film
fitness
flamenco
folk
folklore
folkmetal
folkrock
folktronica
forró
frankreich
französisch
french
funk
future
gangsta
garage
german
ghazal
gitarre
glitch
gospel
gothic
grime
grunge
gypsy
handsup
hard'n'heavy
hardcore
hardstyle
hardtechno
hiphop
historisch
holiday
horror
house
idm
independent
india

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por `'unknown'`. Pero todavía tenemos que ver si los valores ausentes en `'genre'` afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

Ahora ya podemos continuar probando las hipótesis. 

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Comprobaremos esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

* **Dividiremos a los usuarios en grupos por ciudad.**
* **Compararemos cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.**


In [49]:
# contando las pistas reproducidas en cada ciudad
df.groupby('city').count() 


,userid,track,artist,genre,time,day
city,,,,,,
Shelbyville,18512,18512,18512,18512,18512,18512
Springfield,42741,42741,42741,42741,42741,42741


Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.


Ahora agruparemos los datos por día de la semana y encontraremos el número de pistas reproducidas el **lunes, miércoles y viernes.**


In [51]:
# calculando las pistas reproducidas en cada uno de los tres días
df.groupby('day').count()

,userid,track,artist,genre,city,time
day,,,,,,
Friday,21840,21840,21840,21840,21840,21840
Monday,21354,21354,21354,21354,21354,21354
Wednesday,18059,18059,18059,18059,18059,18059


El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.

Crearemos la función `number_tracks()` para calcular el número de canciones reproducidas en un determinado día y ciudad. Requerirá dos parámetros:
* **día de la semana**
* **nombre de la ciudad**

En la función, utilizaremos una variable para almacenar las filas de la tabla original, donde:
  * el valor de la columna `'day'` es igual al parámetro de día
  * el valor de la columna `'city'` es igual al parámetro de ciudad

Aplicaremos un filtrado consecutivo con indexación lógica.

Después, calcularemos los valores de la columna `'user_id'` en la tabla resultante. Almacenaremos el resultado en la nueva variable.

In [52]:
df


,userid,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
...,...,...,...,...,...,...,...
61248,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
61249,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hiphop,Shelbyville,10:00:00,Monday
61250,C5E3A0D5,Jalopiina,unknow,industrial,Springfield,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [59]:
# <creando la función number_tracks()>
def number_tracks(day, city):
      track_list = df[(df['day'] == day) & (df['city'] == city)]
      track_list_count = track_list['userid'].count()
      return track_list_count    


# la función cuenta las pistas reproducidas en un cierto día y ciudad.
# primero recupera las filas del día deseado de la tabla,
# después filtra las filas de la ciudad deseada del resultado,
# entonces, encuentra el número de valores de 'user_id' en la tabla filtrada,
# y devuelve ese número.

Llamaremos a `number_tracks()` seis veces, cambiando los valores de los parámetros, para que recuperemos los datos de ambas ciudades para cada uno de los tres días.

In [61]:
# el número de canciones reproducidas en Springfield el lunes
number_tracks('Monday','Springfield')

15740

In [62]:
# el número de canciones reproducidas en Shelbyville el lunes
number_tracks('Monday','Shelbyville')

5614

In [63]:
# el número de canciones reproducidas en Springfield el miércoles
number_tracks('Wednesday','Springfield')

11056

In [65]:
# el número de canciones reproducidas en Shelbyville el miércoles
number_tracks('Wednesday','Shelbyville')

7003

In [66]:
# el número de canciones reproducidas en Springfield el viernes
number_tracks('Friday','Springfield')

15945

In [67]:
# el número de canciones reproducidas en Shelbyville el viernes
number_tracks('Friday','Shelbyville')

5895

Haremos una tabla con los datos anteriores:


In [68]:
data = {
    'city':['Springfield', 'Shelbyville'],
    'monday':[15740,5614],
    'wednesday':[11056,7003],
    'friday': [15945,5895]
}
tabla_nueva = pd.DataFrame(data)


In [69]:
# tabla con los resultados
tabla_nueva

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusiones Hipótesis 1**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

In [70]:
# obteniendo la tabla spr_general de las filas de df, 
# donde los valores en la columna 'city' es 'Springfield'
spr_general = df[df['city'] == 'Springfield'] 


In [71]:
spr_general = spr_general.reset_index()

In [73]:
spr_general

,index,userid,track,artist,genre,city,time,day
0,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
1,4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
2,6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
3,7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
4,8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
...,...,...,...,...,...,...,...,...
42736,61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Springfield,21:07:12,Monday
42737,61248,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
42738,61250,C5E3A0D5,Jalopiina,unknow,industrial,Springfield,20:09:26,Friday
42739,61251,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [74]:
# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'
shel_general = df[df['city'] == 'Shelbyville'] 


In [75]:
shel_general = shel_general.reset_index()

In [76]:
shel_general

,index,userid,track,artist,genre,city,time,day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
2,3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
3,5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
4,9,E772D5C0,Pessimist,unknow,dance,Shelbyville,21:20:49,Wednesday
...,...,...,...,...,...,...,...,...
18507,61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Shelbyville,21:14:40,Monday
18508,61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Shelbyville,21:06:50,Monday
18509,61241,29E04611,Bre Petrunko,Perunika Trio,world,Shelbyville,13:56:00,Monday
18510,61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Shelbyville,09:22:13,Monday


Crearemos una función para devolver la información de los 15 géneros más populares de un día determinado en un período entre dos marcas de fecha y hora.

In [82]:
def genre_weekday (df, day, time1, time2):
    genre_df = df[(df['day'] == day) & (df['time'] < time2) & (df['time'] > time1)]
    genre_df_grouped =  genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:15]

Compararemos los resultados de la función `genre_weekday()`para Springfield y Shelbyville el lunes por la mañana (de 7 a 11) y el viernes por la tarde (de 17:00 a 23:00):

In [83]:
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)
genre_weekday(spr_general,'Monday','7:00', '11:00')

Series([], Name: genre, dtype: int64)

In [84]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)
genre_weekday(shel_general,'Monday','7:00', '11:00')

Series([], Name: genre, dtype: int64)

In [85]:
# llamando a la función para el viernes por la tarde en Springfield
genre_weekday(spr_general,'Friday','17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknow         110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [86]:
# llamando a la función para el viernes por la tarde en Shelbyville
genre_weekday(shel_general,'Friday','17:00', '23:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknow          47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Conclusión Hipótesis 2**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo dance y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

Agruparemos la tabla `spr_general` por género y encuentraremos el número de canciones reproducidas de cada género con el método `count()`. Después, ordenaremos el resultado en orden descendente y lo guardaremos en `spr_genres`.

In [91]:
#agrupamos la tabla spr_general por la columna 'genre', 
spr_genres = spr_general.groupby('genre')['genre'].count().sort_values(ascending= False)

Imprime las 10 primeras filas de `spr_genres`:

In [90]:
# imprimiendo las 10 primeras filas de spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Ahora haz lo mismo con los datos de Shelbyville.

Agrupamos la tabla `shel_general` por género y encontramos el número de canciones reproducidas de cada género. Después, ordenamos el resultado en orden descendente y lo guardamos en la tabla `shel_genres`:


In [92]:
shel_genres = shel_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Imprime las 10 primeras filas de `shel_genres`:

In [93]:
# imprimiendo las 10 primeras filas de shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusión Hipotesis 3**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades. 
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas. 

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.

### Nota 
En los proyectos profundos, la investigación supone el estudio de hipótesis estadísticas que es más preciso y cuantitativo. También tenemos en cuenta que no siempre podemos sacar conclusiones sobre una ciudad entera basándonos en datos de una sola fuente.
